# **Customer Churn Analysis Using Deep Learning**

## **Import Libraries**

In [23]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder, StandardScaler

## **Data Preprocessing**

### Read DataFrame

In [3]:
# Read dataset
bank_data = pd.read_csv('Bank_Customers.csv')

### Print DataFrame Head

In [4]:
bank_data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Print DataFrame Tail

In [5]:
bank_data.tail()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1
9999,10000,15628319,Walker,792,France,Female,28,4,130142.79,1,1,0,38190.78,0


### Show if the dataframe contains null values

In [6]:
bank_data.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

### Print DataFrame info

In [7]:
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


### Print DataFrame describe

In [8]:
bank_data.describe(include='all')

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000,10000.000000,10000,10000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
unique,NaN,NaN,2932,NaN,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,Smith,NaN,France,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,32,NaN,5014,5457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,5000.50000,1.569094e+07,NaN,650.528800,NaN,NaN,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,NaN,96.653299,NaN,NaN,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,NaN,350.000000,NaN,NaN,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,NaN,584.000000,NaN,NaN,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,NaN,652.000000,NaN,NaN,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,NaN,718.000000,NaN,NaN,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000


### Data Encoding

In [9]:
bank_copy = bank_data.copy()

In [10]:
encoder = LabelEncoder()
bank_copy['Gender'] = encoder.fit_transform(bank_copy['Gender'])

In [11]:
geo_dummies = pd.get_dummies(bank_copy['Geography'])
geo_dummies

,France,Germany,Spain
0,1,0,0
1,0,0,1
2,1,0,0
3,1,0,0
4,0,0,1
...,...,...,...
9995,1,0,0
9996,1,0,0
9997,1,0,0
9998,0,1,0


In [12]:
bank_copy = bank_copy.join(geo_dummies)

In [13]:
bank_copy.drop(['RowNumber','CustomerId','Geography','Surname'], axis=1, inplace=True)

In [14]:
bank_copy

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1,0,0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1,0,0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1,0,0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0,1,0


In [15]:
X = bank_copy.drop('Exited', axis=1)
y= bank_copy['Exited']

In [16]:
scaler = StandardScaler() 
X = scaler.fit_transform(X)

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y ,test_size=0.2, random_state=0)

## **Model**

### Build a Model

In [18]:
model = Sequential()
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(units=1, activation='sigmoid'))

In [19]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

### Fit Data into Model

In [20]:
model.fit(x_train,y_train,epochs=20)

Epoch 1/20
250/250 [==============================] - 2s 3ms/step - loss: 0.4367 - accuracy: 0.8102
Epoch 2/20
250/250 [==============================] - 1s 3ms/step - loss: 0.3638 - accuracy: 0.8510
Epoch 3/20
250/250 [==============================] - 1s 3ms/step - loss: 0.3445 - accuracy: 0.8595
Epoch 4/20
250/250 [==============================] - 1s 3ms/step - loss: 0.3342 - accuracy: 0.8624
Epoch 5/20
250/250 [==============================] - 1s 2ms/step - loss: 0.3280 - accuracy: 0.8675
Epoch 6/20
250/250 [==============================] - 1s 3ms/step - loss: 0.3243 - accuracy: 0.8695
Epoch 7/20
250/250 [==============================] - 1s 2ms/step - loss: 0.3203 - accuracy: 0.8695
Epoch 8/20
250/250 [==============================] - 1s 2ms/step - loss: 0.3181 - accuracy: 0.8649
Epoch 9/20
250/250 [==============================] - 1s 2ms/step - loss: 0.3128 - accuracy: 0.8724
Epoch 10/20
250/250 [==============================] - 1s 3ms/step - loss: 0.3098 - accuracy: 0.8714

### Evaluate Model

In [21]:
model.evaluate(x_test,y_test)

63/63 [==============================] - 1s 2ms/step - loss: 0.3820 - accuracy: 0.8420


[0.3819810748100281, 0.8420000076293945]

## **Prediction**


* Geography: France
* Credit Score: 600
* Gender: Male
* Age: 40 years old
* Tenure: 3 years
* Balance: \$ 60000
* Number of Products: 2
* Does this customer have a credit card ? Yes
* Is this customer an Active Member: Yes
* Estimated Salary: \$ 50000
* So, should we say goodbye to that customer ?

In [24]:
if model.predict(scaler.transform([[600,1,40,3,60000,2,1,1,50000,1,0,0]])) < 0.5:
    print('You will not lose the customer')

else:
    print('You will lose the customer')

You will not lose the customer
